<a href="https://colab.research.google.com/github/LeeHyungDo/Mongotest/blob/master/HPC_NER%EC%8B%A4%EC%8A%B5%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets==3.5.0
!pip install evaluate
!pip install transformers[torch]
!pip install seqeval

In [ ]:
import os
import re
import numpy as np
from tqdm import tqdm

import evaluate
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

import matplotlib.pyplot as plt

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("beomi/gemma-ko-2b", do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

## 토크나이저 테스트

In [ ]:
test_sentence = "안녕하세요, 반갑습니다."

encode = tokenizer.encode(test_sentence)
token_print = [tokenizer.decode(token) for token in encode]

print(encode)
print(token_print)

[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 119, 102]
['[CLS]', '안', '##녕', '##하', '##세', '##요', ',', '반', '##갑', '##습', '##니다', '.', '[SEP]']


In [ ]:
kor_encode = tokenizer.encode("안녕하세요, 반갑습니다")
eng_encode = tokenizer.encode("Hello world")
kor_decode = tokenizer.decode(kor_encode)
eng_decode = tokenizer.decode(eng_encode)

print(kor_encode)
# [101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 102]
print(eng_encode)
# [101, 31178, 11356, 102]
print(kor_decode)
# [CLS] 안녕하세요, 반갑습니다 [SEP]
print(eng_decode)
# [CLS] Hello world [SEP]

[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 102]
[101, 31178, 11356, 102]
[CLS] 안녕하세요, 반갑습니다 [SEP]
[CLS] Hello world [SEP]


# NER Classification


In [ ]:
dataset = load_dataset("kor_ner")

Generating train split:   0%|          | 0/2928 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/366 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/366 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'annot_text', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 2928
    })
    test: Dataset({
        features: ['text', 'annot_text', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 366
    })
    validation: Dataset({
        features: ['text', 'annot_text', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 366
    })
})

In [ ]:
dataset["test"][0]

{'text': '롯데삼강의 홍보실 전일재씨는 "한꺼번에 여러 가지 맛을 볼 수 있다는 것을 강조, 신세대들의 기호에 맞췄다"고 말한다.',
 'annot_text': '<롯데삼강:OG>의 홍보실 <전일재:PS>씨는 "한꺼번에 여러 가지 맛을 볼 수 있다는 것을 강조, 신세대들의 기호에 맞췄다"고 말한다.',
 'tokens': ['롯데삼강',
  '의',
  '홍보실',
  '전일재',
  '씨',
  '는',
  '"',
  '한꺼번에',
  '여러',
  '가지',
  '맛',
  '을',
  '보',
  'ㄹ',
  '수',
  '있',
  '다',
  '는',
  '것',
  '을',
  '강조',
  ',',
  '신세대',
  '들',
  '의',
  '기호',
  '에',
  '맞추',
  '었',
  '다',
  '"',
  '고',
  '말',
  '하',
  'ㄴ다',
  '.'],
 'pos_tags': [12,
  36,
  31,
  12,
  29,
  39,
  1,
  25,
  8,
  29,
  31,
  28,
  2,
  27,
  29,
  2,
  18,
  27,
  29,
  28,
  31,
  9,
  31,
  10,
  36,
  31,
  7,
  2,
  14,
  19,
  1,
  15,
  31,
  24,
  19,
  38],
 'ner_tags': [2,
  1,
  1,
  6,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
ner_feature = dataset["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['I', 'O', 'B_OG', 'B_TI', 'B_LC', 'B_DT', 'B_PS'], id=None), length=-1, id=None)

In [ ]:
label_names = ner_feature.feature.names
label_names

['I', 'O', 'B_OG', 'B_TI', 'B_LC', 'B_DT', 'B_PS']

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels.replace()
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/2928 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
labels = dataset["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'B-PS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "beomi/gemma-ko-2b", id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="ner_seq_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

history = trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.113647,0.919198,0.928918,0.924032,0.964720


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory ner_seq_classifier/checkpoint-183 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "./ner_seq_classifier/checkpoint-183"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("안녕하세요 대한민국 서울에 사는 홍길동입니다.")

[{'entity_group': 'B_OG',
  'score': 0.9360318,
  'word': '##녕하세요',
  'start': 1,
  'end': 5},
 {'entity_group': 'B_LC',
  'score': 0.8979496,
  'word': '대한민국 서울',
  'start': 6,
  'end': 13},
 {'entity_group': 'B_OG',
  'score': 0.9755044,
  'word': '##는',
  'start': 16,
  'end': 17},
 {'entity_group': 'B_PS',
  'score': 0.8987875,
  'word': '홍길동',
  'start': 18,
  'end': 21},
 {'entity_group': 'B_OG',
  'score': 0.7128355,
  'word': '##니다',
  'start': 22,
  'end': 24}]